# Sparse Hebbian Learning with full homeostasis : testing different rescaling functions

In this notebook, we test the convergence of SparseNet as a function of different parameters tuning the quantization. These parameters only influence the way we select dictionary elements and thus the homeostasis.



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [ ]:
from shl_scripts.shl_experiments import SHL_set
tag = 'rescaling'
opts = dict(homeo_method='HEH', eta_homeo=0.05, C=5., nb_quant=256, P_cum=None, verbose=1)
experiments = SHL_set(opts, tag=tag)
list_figures = ['show_dico']
list_figures = []

In [ ]:
!ls -l {experiments.shl.data_cache}/{tag}*

We will use the ``joblib`` package do distribute this computation on different CPUs.

## different rescaling values

In [ ]:
experiments.scan(variable='C', list_figures=list_figures, display='')
fig, ax = experiments.scan(variable='C', list_figures=[], display='final')

## different quantization parameters

with a fixed $C=5.$

In [ ]:
experiments.scan(variable='nb_quant', list_figures=list_figures, display='')
fig, ax = experiments.scan(variable='nb_quant', list_figures=[], display='final')

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts